# Анализ результатов A/A/B-тестирования

Данное исследование представляет собой анализ результатов эксперимента, целью которого было проверить, не скажется ли смена шрифтов в приложении негативно на активности пользователей. Эксперимент проводился на базе A/A/B-тестирования.

**Структура данных:**

Данные о результатах A/A/B эксперимента хранятся в файле `logs_exp.csv`:

* `EventName` – название события


* `DeviceIDHash` – уникальный идентификатор пользователя


* `EventTimestamp` – Unix-время


* `ExpId` – номер эксперимента: 246 и 247 - контрольные группы, 248 - экспериментальная


**План:**

<div class="toc">
   <ul class="toc-item">
      <li><span><a href="#Setup" data-toc-modified-id="Setup-2">Setup</a></span></li>
      <li>
         <span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-3">Предобработка данных</a></span>
         <ul class="toc-item">
            <li><span><a href="#Преобразование-типов-столбцов" data-toc-modified-id="Преобразование-типов-столбцов-3.1">Преобразование типов столбцов</a></span></li>
            <li><span><a href="#Удаление-дубликатов" data-toc-modified-id="Удаление-дубликатов-3.2">Удаление дубликатов</a></span></li>
            <li><span><a href="#Удаление-неполных-данных" data-toc-modified-id="Удаление-неполных-данных-3.3">Удаление неполных данных</a></span></li>
         </ul>
      </li>
      <li>
         <span><a href="#Анализ-результатов-эксперимента" data-toc-modified-id="Анализ-результатов-эксперимента-4">Анализ результатов эксперимента</a></span>
         <ul class="toc-item">
            <li><span><a href="#Воронки-событий" data-toc-modified-id="Воронки-событий-4.1">Воронки событий</a></span></li>
            <li><span><a href="#Количество-событий-на-пользователя" data-toc-modified-id="Количество-событий-на-пользователя-4.2">Количество событий на пользователя</a></span></li>
         </ul>
      </li>
      <li><span><a href="#Итоги" data-toc-modified-id="Итоги-5">Итоги</a></span></li>
   </ul>
</div>

# Setup

Для того чтобы писать меньше кода, я импортирую [свои собственные шаблоны](https://gist.github.com/rusmux/a74e5060a470f45e0f2d52b2484a1ecf) для plotly.

In [ ]:
%%bash
if ! test -f "custom_plotly_templates.py"; then
    wget https://gist.github.com/rusmux/a74e5060a470f45e0f2d52b2484a1ecf/raw/369401538ba9f7909b4342288044adfedcbfe1aa/custom_plotly_templates.py &> /dev/null
    echo "Successfully downloaded custom plotly templates."
fi

In [ ]:
import re

import ipywidgets as widgets

import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from scipy import stats

import custom_plotly_templates
from custom_plotly_templates import show_config

In [ ]:
pd.set_option("display.float_format", "{:.2f}".format)

In [ ]:
pio.templates.default = "plotly_white+custom_white"
pio.renderers["notebook"].config.update(show_config)

# Предобработка данных

In [ ]:
data = pd.read_csv("logs_exp.csv", sep="\t")
data

Сначала приведем названия столбцов к snake стилю.

In [ ]:
from_pascal_to_snake = lambda name: "_".join(re.findall("[A-Z][^A-Z]*", name)).lower()
columns = list(map(from_pascal_to_snake, data.columns))
columns[1] = "device_id_hash"  # device_i_d_hash
data.columns = columns

In [ ]:
data.info(memory_usage="deep")

Явных пропусков нет. Проверим на неявные.

In [ ]:
data.isin([0, "", " ", "None"]).sum()

Неявных пропусков тоже нет. Преобразуем типы столбцов.

## Преобразование типов столбцов

In [ ]:
data["device_id_hash"].nunique()

Количество уникальных пользователей 7551 — значительно меньше общего числа записей, так что имеет смысл конвертировать столбец в тип `category`.

In [ ]:
data["device_id_hash"] = pd.Categorical(data["device_id_hash"])

In [ ]:
data["event_name"].unique().tolist()

Различных событий всего 5, так что тоже сконвертируем в тип `category` и приведем к snake стилю.

In [ ]:
data["event_name"] = pd.Categorical(data["event_name"].apply(from_pascal_to_snake))

In [ ]:
data["exp_id"].unique().tolist()

Присвоим экспериментам 246 и 247 группы A1 и A2 соответственно, а эксперименту 248 группу B. Сохраним идентификаторы экспериментов и соответствующие им группы в отдельной таблице.

In [ ]:
group_exp_ids = pd.DataFrame({"group": ["A1", "A2", "B"]}, index=pd.Index([246, 247, 248], name="exp_id"))
group_exp_ids

In [ ]:
data = data.merge(group_exp_ids, right_index=True, left_on="exp_id").drop("exp_id", axis=1)
data["group"] = data["group"].astype("category")

Переведем столбец `event_timestamp` из секунд в формат даты и времени.

In [ ]:
data["event_dt"] = pd.to_datetime(data["event_timestamp"], unit="s")
data.drop("event_timestamp", axis=1, inplace=True)

In [ ]:
data.sample(5)

In [ ]:
data.info(memory_usage="deep")

В результате преобразований количество потребляемой памяти снизилось в ~4.5 раза.

## Удаление дубликатов

Проверим данные на наличие дубликатов.

In [ ]:
print("Количество дубликатов:", data.duplicated().sum())
data[data.duplicated(keep=False)]

В данных присутствует 413 полных дубликатов. Вероятно, это ошибка логирования. Удалим их.

In [ ]:
data.drop_duplicates(inplace=True)

Посмотрим на неявные дубликаты, но прежде проверим, что каждому пользователю соответствует только одна группа. Создадим словарь с id пользователя и его группой.

In [ ]:
user_groups = data[["device_id_hash", "group"]].drop_duplicates().set_index("device_id_hash")
assert len(user_groups) == data["device_id_hash"].nunique()

Да, каждому пользователю соответствует только одна группа.

In [ ]:
data[data.duplicated(subset=["device_id_hash", "event_dt"], keep=False)]

Неявных дубликатов гораздо больше – порядка 40 тысяч. Однако вполне возможно, что в одну секунду пользователь находился в корзине, а после оплаты его перенаправили на страницу с сообщением об успешной оплате. Так что не будем ничего удалять.

## Удаление неполных данных

In [ ]:
event_names = [
    "main_screen_appear",
    "offers_screen_appear",
    "cart_screen_appear",
    "payment_screen_successful",
    "tutorial",
]

event_names_ru = [
    "Главная страница",
    "Предложение продукта",
    "Переход в корзину",
    "Успешная оплата",
    "Обучение",
]

event_names_map = dict(zip(event_names, event_names_ru))

In [ ]:
def get_event_dts_distribution(event="all", group="all", interactive=False):
    def get_fig_data(event="all", group="all"):
        fig_data = data
        if event != "all":
            fig_data = fig_data.query("event_name == @event")
        if group != "all":
            if group == "A общая":
                fig_data = fig_data.query("group in ['A1', 'A2']")
            else:
                fig_data = fig_data.query("group == @group")
        return fig_data

    fig_data = get_fig_data(event, group)
    fig = px.histogram(fig_data, x="event_dt", nbins=400)

    fig.update_layout(
        title="Количество событий в зависимости от даты",
        xaxis_title=None,
        yaxis_title=None,
        xaxis_showgrid=True,
        yaxis_showgrid=True,
        xaxis_linecolor="black",
        yaxis_linecolor="black",
        xaxis_rangeslider_visible=True,
        xaxis_range=(data["event_dt"].min(), data["event_dt"].max()),
        yaxis_range=(0, 3000),
        height=550,
        margin=dict(t=70),
    )

    if interactive:

        fig = go.FigureWidget(fig)

        def update(event, group):
            fig_data = get_fig_data(event, group)

            with fig.batch_update():
                fig.data[0].x = fig_data["event_dt"]

        event_options = [("Все", "all"), *list(zip(event_names_ru, event_names))]
        event_widget = widgets.Dropdown(options=event_options, value=event, description=" событие")
        group_widget = widgets.Dropdown(
            options=[("Все", "all"), ("A1",) * 2, ("A2",) * 2, ("A общая",) * 2, ("B",) * 2],
            value=group,
            description="группа",
        )
        widgets.interactive_output(update, dict(event=event_widget, group=group_widget))

        ui = widgets.HBox([event_widget, group_widget], layout=widgets.Layout(margin="10px 0px 0px 0px"))
        display(ui)

    return fig

In [ ]:
get_event_dts_distribution(interactive=True)

In [ ]:
print("From:", data["event_dt"].min().strftime("%X, %b %d, %Y"))
print("To:  ", data["event_dt"].max().strftime("%X, %b %d, %Y"))

Данные с конца июля по начало августа 2019 года, но высокая пользовательская активность началась только с августа, вероятно, там просто неполные данные. Возможно, эксперимент начали в конце июля, и неделю уделили на проверку, что логирование работает правильно. После окончания проверки, уже начали логировать полностью, но некоторые июльские события пришли с задержкой и оказались в рабочем логировании. В любом случае удалим их. События идут волнами, циклично, потому что ночью мало кто заходит на сайт. Распределения не отличаются для различных событий и групп.

In [ ]:
len_before = len(data)
data = data.query("event_dt >= '2019-08-01 00:00:00'")
print("Минимальные дата и время:", data["event_dt"].min().strftime("%X, %b %d, %Y"))

In [ ]:
print(f"Потеря данных: {1 - len(data) / len_before:.2%}")

In [ ]:
data["device_id_hash"].nunique()

После удаления неполных данных, количество пользователей сократилось на 17. Однако эти пользователи все равно входят в группы, так что не будем их удалять, хоть у них и нет событий в августе.

In [ ]:
print(f"Потеря пользователей: {17/7551:.2%}")

## Результаты предобработки

Был дан протокол A/A/B-эксперимента за промежуток времени с 25 июля по 7 августа 2019 года. Для удобства работы и уменьшения потребления памяти, столбцы были преобразованы в другие типы данных. Были удалены дублирующие друг друга записи, а также отброшены неполные данные за июль. Потеря данных составила 1.16%, а потеря пользователей — 0.23%.

# Анализ результатов эксперимента

In [ ]:
data

In [ ]:
data.info()

In [ ]:
def get_event_names_distribution():
    fig = px.histogram(
        y=data["event_name"].map(event_names_map).str.capitalize(), text_auto=".2p", histnorm="probability"
    )

    fig.update_layout(
        title="Распределение событий в данных",
        title_x=0.5,
        title_y=0.94,
        margin_t=80,
        xaxis_title=None,
        yaxis_title=None,
        xaxis_showticklabels=False,
        yaxis_linecolor="black",
        yaxis_categoryorder="total ascending",
    )

    return fig

In [ ]:
get_event_names_distribution()

Событий всего 5 и половиной событий является посещение главной страницы. Обучение составляет меньше процента.

In [ ]:
def get_users_distribution():
    fig = px.histogram(
        user_groups,
        x="group",
        text_auto=".2p",
        histnorm="probability",
        category_orders={"group": ["A1", "A2", "B"]},
    )

    fig.update_layout(
        title="Распределение пользователей по группам",
        title_x=0.5,
        title_y=0.94,
        margin_t=80,
        xaxis_title=None,
        yaxis_title=None,
        xaxis_linecolor="black",
        yaxis_showticklabels=False,
        width=600,
        height=425,
    )

    return fig

In [ ]:
get_users_distribution()

Группы распределены практически одинаково. Создадим таблицу с количеством событий для каждого пользователя.

## Воронки событий

In [ ]:
user_events = (
    data.pivot_table(index="device_id_hash", columns="event_name", values="event_dt", aggfunc=len)
    .join(user_groups, how="right")
    .fillna(0, axis=1)[event_names + ["group"]]
    .apply(pd.to_numeric, downcast="unsigned", errors="ignore")
)

user_events

In [ ]:
data

In [ ]:
user_events.drop("group", axis=1).astype(bool).agg(["mean", "sum"]).T

Не все пользователи увидели главный экран, а обучение прошли всего 840 человек — 11% всех пользователей. Главный экран могли пропустить пользователи, которые сохранили страницу с предложениями в закладки. Посмотрим на них. 

In [ ]:
user_events.query("main_screen_appear == 0").drop("group", axis=1).astype(bool).agg(["mean", "sum"]).T

Действительно, 74% таких пользователей совершили покупки. Так что, вероятно, они просто сохранили страницу с предложениями в закладки. Таких пользователей мало, так что не будем учитывать их при построении воронки событий. Посмотрим на тех, кто прошел туториал.

In [ ]:
user_events.query("tutorial > 0").drop("group", axis=1).astype(bool).agg(["mean", "sum"]).T

У тех, кто прошел обучение чуть выше конверсия на каждом шаге воронки от появления главного экрана до сообщения об успешной оплаты. Однако самих людей, кто прошел это обучение относительно мало. Если предполагается, что только что пришедший пользователь должен пройти обучение, то это либо проблема с технической стороны, либо данные пользователи уже все прошли это обучение. Во всяком случае, не будем включать это событие в воронку событий.

In [ ]:
user_events_bool = user_events.astype(bool)
user_events_bool["group"] = user_events["group"]

event_funnel = (
    user_events_bool.query("main_screen_appear == True")
    .melt(id_vars="group", var_name="event")
    .pivot_table(index="event", columns="group", values="value", aggfunc="sum")
    .loc[event_names[:-1]]
)

event_funnel

In [ ]:
def get_group_funnels(
    event_funnel, *, value_mode="percent initial", merge_A_groups=False, interactive=False, **kwargs
):

    title = kwargs.get("title", "Воронка событий по группам")

    def get_group_funnels_(
        event_funnel,
        *,
        value_mode="percent initial",
        merge_A_groups=False,
        interactive=False,
        value_mode_widget=None,
        **kwargs,
    ):
        title = kwargs.get("title", "Воронка событий по группам")

        event_funnel_ru = event_funnel.copy()
        event_funnel_ru.index = event_funnel_ru.index.map(event_names_map).str.capitalize()

        if merge_A_groups:
            event_funnel_ru.insert(2, "A", event_funnel_ru["A1"] + event_funnel_ru["A2"])
            event_funnel_ru.drop(["A1", "A2"], axis=1, inplace=True)

        funnels = px.funnel(event_funnel_ru).data

        fig = make_subplots(
            1, len(funnels), shared_yaxes=True, column_titles=event_funnel_ru.columns.tolist()
        )

        for i, funnel in enumerate(funnels, start=1):
            fig.add_trace(funnel, 1, i)

        fig.update_layout(
            title=title, title_x=0.55, title_y=0.96, yaxis_ticklen=10, showlegend=False, height=525
        )

        fig.update_traces(textinfo=value_mode, insidetextfont_color="white")

        if interactive:

            fig = go.FigureWidget(fig)

            def update(value_mode="percent initial"):
                with fig.batch_update():
                    fig.update_traces(textinfo=f"{value_mode}")

            widgets.interactive_output(update, dict(value_mode=value_mode_widget))

            display(fig)

        else:
            return fig

    if interactive:

        value_modes = [
            ("Процент абсолютный", "percent initial"),
            ("Процент относительный", "percent previous"),
            ("Абсолютное значение", "value"),
        ]
        value_mode_widget = widgets.Dropdown(
            options=value_modes,
            value=value_mode,
            description="тип значения",
            style=dict(description_width="initial"),
        )

        merge_A_groups_widget = widgets.Checkbox(
            value=merge_A_groups,
            description="объединить группы A",
            indent=False,
            layout=widgets.Layout(margin="3px 0px 0px 20px"),
        )

        output = widgets.interactive_output(
            get_group_funnels_,
            dict(
                event_funnel=widgets.fixed(event_funnel),
                value_mode=widgets.fixed(value_mode),
                merge_A_groups=merge_A_groups_widget,
                interactive=widgets.fixed(interactive),
                value_mode_widget=widgets.fixed(value_mode_widget),
                title=widgets.fixed(title),
            ),
        )

        ui = widgets.HBox(
            [value_mode_widget, merge_A_groups_widget], layout=widgets.Layout(margin="10px 0px 0px 20px")
        )
        display(ui, output)

    else:
        fig = get_group_funnels_(event_funnel, value_mode, merge_A_groups, interactive, title=title)
        return fig

In [ ]:
get_group_funnels(event_funnel, interactive=True)

Воронки очень похожи между собой, и группа B не лучше/хуже контрольных групп.

До успешной оплаты доходит в среднем 47% процентов всех пользователей. Самая низкая конверсия на стадии перехода с главной страницы на страницу с предложениями продукта — 62%. А если уже пользователь перешел на эту страницу, то на оставшихся этапах воронки конверсия перехода выше 80%.

Проверим, есть ли статистически значимая разница между группами. Нулевой гипотезой будет, что конверсия пользователей в покупателей для групп не отличается, а альтернативной — что конверсии различны. За критический уровень p-значения примем 0.05.

In [ ]:
def get_group_pvalues(user_events, event=None, interactive=False):
    def get_group_pvalues_(user_events, event):
        event_A1 = user_events.query("group == 'A1'")[event]
        event_A2 = user_events.query("group == 'A2'")[event]
        event_A = pd.concat([event_A1, event_A2])
        event_B = user_events.query("group == 'B'")[event]

        print("P-values:\n")
        print("A1 vs A2:", round(stats.ttest_ind(event_A1, event_A2, equal_var=True).pvalue, 3))
        print("A1 vs  B:", round(stats.ttest_ind(event_A1, event_B).pvalue, 3))
        print("A2 vs  B:", round(stats.ttest_ind(event_A2, event_B).pvalue, 3))
        print("A  vs  B:", round(stats.ttest_ind(event_A, event_B).pvalue, 3))

    if interactive:
        if not event:
            event = "payment_screen_successful"
        event_options = list(zip(event_names_ru, event_names))
        event_widget = widgets.Dropdown(
            options=event_options,
            value=event,
            description="событие",
            layout=widgets.Layout(margin="10px 0px 20px 0px"),
        )
        widgets.interact(get_group_pvalues_, user_events=widgets.fixed(user_events), event=event_widget)
    else:
        if not event:
            raise ValueError("If 'interactive' is False, parameter 'event' should be provided")
        get_group_pvalues_(user_events, event)

In [ ]:
from statsmodels.stats.weightstats import ztest

event = "payment_screen_successful"
event_A1 = user_events_bool.query("group == 'A1'")[event]
event_A2 = user_events_bool.query("group == 'A2'")[event]
event_A = pd.concat([event_A1, event_A2])
event_B = user_events_bool.query("group == 'B'")[event]

print("P-values:\n")
print("A1 vs A2:", round(ztest(event_A1, event_A2)[1], 3))
print("A1 vs  B:", round(ztest(event_A1, event_B)[1], 3))
print("A2 vs  B:", round(ztest(event_A2, event_B)[1], 3))
print("A  vs  B:", round(ztest(event_A, event_B)[1], 3))

In [ ]:
get_group_pvalues(user_events_bool, interactive=True)

Между никакими группами нет статистически значимых различий. Посмотрим на воронку тех, кто проходил обучение.

In [ ]:
user_events_bool_tutorial = user_events.query("tutorial > 0").astype(bool)
user_events_bool_tutorial["group"] = user_events["group"]

event_funnel_tutorial = (
    user_events_bool_tutorial.melt(id_vars="group", var_name="event")
    .pivot_table(index="event", columns="group", values="value", aggfunc="sum")
    .loc[event_names[:-1]]
)

In [ ]:
get_group_funnels(
    event_funnel_tutorial, interactive=True, title="Воронка событий для тех, кто прошел обучение"
)

Тут конверсии у группы B чуть выше, чем у контрольных групп. Проверим, есть ли статистическая значимость у этой разницы.

In [ ]:
get_group_pvalues(user_events_bool_tutorial, interactive=True)

С группой А2 вероятность такой разницы меньше 5%, но с группой А1 она уже составляет почти 30%. В целом с контрольными группами вероятность выше порога в 5%, так что нельзя сказать, что между группами есть статистически значимая разница.

In [ ]:
user_events_bool_no_main_screen = user_events.query("main_screen_appear == 0").astype(bool)
user_events_bool_no_main_screen["group"] = user_events["group"]

event_funnel_no_main_screen = (
    user_events_bool_no_main_screen.melt(id_vars="group", var_name="event")
    .pivot_table(index="event", columns="group", values="value", aggfunc="sum")
    .loc[event_names[1:-1]]
)

In [ ]:
get_group_funnels(
    event_funnel_no_main_screen,
    interactive=True,
    title="Воронка событий для тех, кто не посещал главную страницу",
)

Тут тоже есть небольшое различие между группами, но в выборках слишком мало человек, чтобы сказать, что между ними есть статистически значимая разница.

In [ ]:
get_group_pvalues(user_events_bool_no_main_screen, interactive=True)

Как и ожидалось. Посмотрим на распределение количества событий на пользователя. Ведь может быть так, что количество конвертировавшихся пользователей одинаково, но в группе B они совершают больше покупок.

## Количество событий на пользователя

In [ ]:
def get_user_events_distribution(*, event="all", group="all", nonzero=False, interactive=False):

    if not interactive and not event:
        raise ValueError("If 'interactive' is False, parameter 'event' should be provided")
    if interactive and not event:
        event = "main_screen_appear"

    def get_fig_data(event="all", group="all", nonzero=False):
        fig_data = user_events
        if group != "all":
            if group == "A общая":
                fig_data = fig_data.query("group in ['A1', 'A2']")
            else:
                fig_data = fig_data.query("group == @group")
        if event == "all":
            fig_data = fig_data.sum(axis=1, numeric_only=True)
        else:
            fig_data = fig_data[event]
        if nonzero:
            fig_data = fig_data[fig_data > 0]
        return fig_data

    def get_title(event="all"):
        if event == "all":
            return f"Распределение количества событий на одного пользователя"
        return (
            f'Распределение количества событий "{event_names_map[event].capitalize()}" на одного пользователя'
        )

    fig_data = get_fig_data(event, group).astype(int)  # plotly can't handle uint16
    fig = px.histogram(x=fig_data, title=get_title(event), marginal="box", histnorm="probability")

    fig.update_layout(
        xaxis_title=None,
        yaxis_title=None,
        xaxis_linecolor="black",
        yaxis_linecolor="black",
        xaxis_showgrid=True,
        yaxis_showgrid=True,
        xaxis_rangeslider_visible=True,
        xaxis_rangemode="nonnegative",
        yaxis_tickformat=".2p",
        height=550,
    )

    fig.data[0].update(xbins_size=1)

    if interactive:

        fig = go.FigureWidget(fig)

        def update(event="all", group="all", xbins_size=1, nonzero=False):
            fig_data = get_fig_data(event, group, nonzero).astype(int)
            with fig.batch_update():
                fig.data[0].x = fig_data
                fig.data[0].update(xbins_size=xbins_size)
                fig.data[1].x = fig_data
                fig.layout.title.text = get_title(event)

        event_options = [("Все", "all"), *list(zip(event_names_ru, event_names))]
        event_widget = widgets.Dropdown(options=event_options, value=event, description="событие")
        group_widget = widgets.Dropdown(
            options=[("Все", "all"), ("A1",) * 2, ("A2",) * 2, ("A общая",) * 2, ("B",) * 2],
            value=group,
            description="группа",
        )
        xbins_size_widget = widgets.IntSlider(
            1,
            1,
            10,
            description="размер корзины",
            style=dict(description_width="initial"),
            layout=widgets.Layout(margin="2px 0px 0px 40px"),
        )

        nonzero_widget = widgets.Checkbox(
            value=nonzero, description="больше нуля", layout=widgets.Layout(margin="10px 0px 0px 0px")
        )

        widgets.interactive_output(
            update,
            dict(
                event=event_widget, group=group_widget, xbins_size=xbins_size_widget, nonzero=nonzero_widget
            ),
        )

        horizontal_box = widgets.HBox([event_widget, group_widget, xbins_size_widget])
        ui = widgets.VBox([horizontal_box, nonzero_widget], layout=widgets.Layout(margin="10px 0px 0px 0px"))

        display(ui)

    return fig

In [ ]:
get_user_events_distribution(event="payment_screen_successful", interactive=True)

В целом, все распределения похожи на распределение Пуассона, а разницы между группами незаметно. Медианное количество событий на пользователя равно 20, но есть пользователи, у которых было более 1000 событий за эти 14 дней. По медиане один покупатель делает 4 покупки.

In [ ]:
user_events.query("payment_screen_successful > 500")

Некоторые пользователи с начала августа успели совершить более 500 покупок, причем они относительно мало заходили на главную страницу. Возможно, это какие-нибудь боты. Посмотрим поближе.

In [ ]:
def get_user_timeline(device_id_hash):

    fig_data = data.query(f"device_id_hash == {device_id_hash}").copy()
    fig_data["event_name"] = fig_data["event_name"].astype(str).map(event_names_map).str.capitalize()

    fig = px.scatter(
        fig_data,
        x="event_dt",
        y="event_name",
        color="event_name",
        title=f'Временная линия пользователя<br><sup style="color:grey;text-align:center;">{device_id_hash}</sup>',
    )

    fig.update_layout(
        xaxis_title=None,
        yaxis_title=None,
        xaxis_linecolor="black",
        yaxis_linecolor="black",
        xaxis_rangeslider_visible=True,
        yaxis_showgrid=True,
        xaxis_range=(
            fig_data["event_dt"].min() - pd.Timedelta(5000, unit="sec"),
            fig_data["event_dt"].max() + pd.Timedelta(5000, unit="sec"),
        ),
        showlegend=False,
        height=450,
        margin=dict(t=100),
        title_y=0.93,
    )

    return fig

In [ ]:
get_user_timeline("6304868067479728361")

Данный пользователь с 12 до 3 часов ночи, не переставая, совершал покупки, и за 7 дней в сумме совершил 1085 покупок. Очень интересно. Сразу возникает вопрос, а не поддерживает ли магазин покупку сразу множество товаров/услуг? Особенно, если кто-то пытается купить твой товар 1085 раз. Может, это, конечно, какой-нибудь магазин одежды, который в это время выпустил уникальную линейку одежды, а кто-то занимался ритейлом и скупал ее. Продолжим изучать группы.

In [ ]:
def get_user_events_distribution_by_groups(
    event="all", *, merge_A_groups=False, nonzero=False, interactive=False
):

    if not interactive and not event:
        raise ValueError("If 'interactive' is False, parameter 'event' should be provided")
    if interactive and not event:
        event = "main_screen_appear"

    def get_user_events_distribution_by_groups_(
        event="all",
        *,
        merge_A_groups=False,
        nonzero=False,
        interactive=False,
        event_widget=None,
        nonzero_widget=None,
    ):

        fig_data = user_events.copy().astype(int, errors="ignore")
        fig_data["all"] = fig_data.sum(axis=1, numeric_only=True)

        if merge_A_groups:
            fig_data["group"] = fig_data["group"].astype(str).replace("A1|A2", "A", regex=True)

        def get_fig_data(event, nonzero=False):
            fig_data_ = fig_data[[event, "group"]]
            if nonzero:
                fig_data_ = fig_data_[fig_data_[event] > 0]
            return fig_data_

        def get_title(event="all"):
            if event == "all":
                return f"Распределение количества событий на одного пользователя"
            return f'Распределение количества событий "{event_names_map[event].capitalize()}" на одного пользователя'

        fig = px.histogram(
            get_fig_data(event, nonzero),
            color="group",
            histnorm="probability",
            barmode="overlay",
            title=get_title(event),
        )

        fig.update_layout(
            xaxis_title=None,
            yaxis_title=None,
            xaxis_showgrid=True,
            yaxis_showgrid=True,
            xaxis_linecolor="black",
            yaxis_linecolor="black",
            xaxis_rangeslider_visible=True,
            xaxis_range=(0, 100),
            yaxis_tickformat=".2p",
            legend_title="Группа<br>",
            legend_tracegroupgap=10,
            height=550,
            margin=dict(t=70),
            title_y=0.97,
        )

        fig.update_traces(xbins_size=1)

        if interactive:

            fig = go.FigureWidget(fig)

            def get_trace_data(event, group, nonzero=False):
                trace_data = fig_data.query("group == @group")[event]
                if nonzero:
                    trace_data = trace_data[trace_data > 0]
                return trace_data

            def update(event, nonzero=False):
                with fig.batch_update():
                    fig.for_each_trace(
                        lambda trace: trace.update(x=get_trace_data(event, trace.legendgroup, nonzero))
                    )
                    fig.layout.title.text = get_title(event)

            widgets.interactive_output(update, dict(event=event_widget, nonzero=nonzero_widget))
            display(fig)

        else:
            return fig

    if interactive:

        event_options = [("Все", "all"), *list(zip(event_names_ru, event_names))]
        event_widget = widgets.Dropdown(options=event_options, value=event, description=" событие")

        merge_A_groups_widget = widgets.Checkbox(
            value=merge_A_groups,
            description="объединить группы A",
            indent=False,
            layout=widgets.Layout(margin="3px 0px 0px 20px"),
        )

        nonzero_widget = widgets.Checkbox(
            value=nonzero, description="больше нуля", layout=widgets.Layout(margin="10px 0px 0px 0px")
        )

        output = widgets.interactive_output(
            get_user_events_distribution_by_groups_,
            controls=dict(
                event=widgets.fixed(event),
                merge_A_groups=merge_A_groups_widget,
                nonzero=widgets.fixed(nonzero),
                interactive=widgets.fixed(interactive),
                event_widget=widgets.fixed(event_widget),
                nonzero_widget=widgets.fixed(nonzero_widget),
            ),
        )

        horizontal_box = widgets.HBox([event_widget, merge_A_groups_widget])
        ui = widgets.VBox([horizontal_box, nonzero_widget], layout=widgets.Layout(margin="10px 0px 0px 0px"))
        display(ui, output)

    else:
        fig = get_user_events_distribution_by_groups_(
            event=event, merge_A_groups=merge_A_groups, nonzero=nonzero
        )
        return fig

In [ ]:
get_user_events_distribution_by_groups(
    event="payment_screen_successful", merge_A_groups=True, interactive=True
)

In [ ]:
get_group_pvalues(user_events, interactive=True)

Распределения идентичные, и пользователи группы B совершают такое же количество покупок, как и пользователи контрольных групп. Возможно, пользователи экспериментальной группы в среднем делают первую покупку раньше. Проверим это, посмотрим на распределение часов с момента первого события до того, когда была сделана покупка.

In [ ]:
data["lifetime"] = data.groupby("device_id_hash")["event_dt"].transform(lambda group: group - group.min())
data["lifetime_hours"] = data["lifetime"] / pd.Timedelta(hours=1)

In [ ]:
def get_payment_lifetimes_distribution_by_groups(*, merge_A_groups=False, interactive=False):
    def get_payment_lifetimes_distribution_by_groups_(merge_A_groups=False):

        fig_data = data.query("event_name == 'payment_screen_successful'").copy()
        if merge_A_groups:
            fig_data["group"] = fig_data["group"].astype(str).replace("A1|A2", "A", regex=True)

        fig = px.histogram(
            fig_data,
            x="lifetime_hours",
            color="group",
            histnorm="probability",
            barmode="overlay",
            title="Распределение среднего количества часов до покупки",
            category_orders={"group": ["A", "B"] if merge_A_groups else ["A1", "A2", "B"]},
        )

        fig.update_layout(
            xaxis_title=None,
            yaxis_title=None,
            xaxis_showgrid=True,
            yaxis_showgrid=True,
            xaxis_linecolor="black",
            yaxis_linecolor="black",
            legend_title="Группа<br>",
            legend_tracegroupgap=10,
            yaxis_tickformat=".2p",
            title_y=0.96,
            margin=dict(t=70),
        )

        return fig

    if interactive:

        merge_A_groups_widget = widgets.Checkbox(
            value=merge_A_groups,
            description="объединить группы A",
            indent=False,
            layout=widgets.Layout(margin="10px 0px 0px 60px"),
        )
        widgets.interact(get_payment_lifetimes_distribution_by_groups_, merge_A_groups=merge_A_groups_widget)

    else:
        fig = get_payment_lifetimes_distribution_by_groups_(merge_A_groups)
        return fig

In [ ]:
get_payment_lifetimes_distribution_by_groups(interactive=True)

В целом распределения похожи — нет существенной разницы во времени совершения покупок.

# Итоги

Был дан протокол A/A/B-эксперимента за промежуток времени с 25 июля по 7 августа 2019 года. Для удобства работы и уменьшения потребления памяти, столбцы были преобразованы в другие типы данных. Были удалены дублирующие друг друга записи, а также отброшены неполные данные за июль. Потеря данных составила 1.16%, а потеря пользователей — 0.23%.

При проверке гипотез критическим p-значением было выбрано 0.05. Было проведено 18 тестов, что дает вероятность ~60% `(1-0.95^18)` неправильного отвержения нулевой гипотезы. Однако ни одна нулевая гипотеза не была отвергнута.

Между группами **не было найдено статистически значимой разницы** в воронке событий. 

В целом по группам:

* До успешной оплаты доходит в среднем 47% процентов всех пользователей.


* Самая низкая конверсия на стадии перехода с главной страницы на страницу с предложениями продукта — 62%.


* Если уже пользователь перешел на эту страницу, то на оставшихся этапах воронки конверсия перехода выше 80%.

Группы также **не отличаются** в количестве событий на пользователя — группы совершают одинаковое количество покупок.

В целом по группам:

* Половина пользователей имеет больше 20 событий.


* Половина покупателей совершает более 4 покупок.

В итоге можно сказать, что смена шрифтов в приложении никак не повлияла на поведение пользователей.